In [1]:
%matplotlib inline
from __future__ import print_function
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
#from sklearn.cross_validation import cross_val_score, KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix
from scipy.stats import sem
from sklearn.preprocessing import PolynomialFeatures
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectFromModel

# Contains linear models, e.g., linear regression, ridge regression, LASSO, etc.
import sklearn.linear_model as sklin
# Provides train-test split, cross-validation, etc.
import sklearn.cross_validation as skcv
# Provides grid search functionality
import sklearn.grid_search as skgs
# For data normalization
import sklearn.preprocessing as skpr
# Allows us to create custom scoring functions
import sklearn.metrics as skmet
from sklearn import grid_search

In [2]:
def eval_pred(predictor, X, y, K=3):
    #scores = cross_val_score(pred, X, y, cv = 3)
    scores = skcv.cross_val_score(predictor, X, y, scoring='accuracy', cv=5)
    print('C-V score =', np.mean(scores), '+/-', np.std(scores))

In [3]:
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

In [4]:
def score(gtruth, pred):
    diff = gtruth - pred
    return np.sqrt(np.mean(np.square(diff)))

In [5]:
X = pd.read_csv('train.csv')
del X['Id']
y = X.y
del X['y']
pd.DataFrame(X).head()
X_test = pd.read_csv('test.csv')
del X_test['Id']

print(X.shape, y.shape, X_test.shape)


(1000, 15) (1000,) (3000, 15)


In [6]:
#normalise the data
scaler= skpr.StandardScaler(with_mean=True, with_std=True, copy=True)
X_scaled = scaler.fit_transform(X)




# build pipeline to train


a = [100]




polynomial_features = PolynomialFeatures(include_bias=True)

svm = SVC(C=1.0, kernel='linear', gamma='auto', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, 
          class_weight=None, verbose=True, max_iter=-1, decision_function_shape='ovr', random_state=None)
sfm = SelectFromModel(svm, prefit=False)

estimators = Pipeline([('pf', polynomial_features),
                             ('feature_selection', sfm),('svm', svm)])
parameters = {'svm__C':np.logspace(-3,3,7),#'svm__degree':[2,3,4],
              'feature_selection__threshold':np.linspace(0.3,0.9,8),'pf__degree':[2], }


clf = grid_search.GridSearchCV(estimators, param_grid=parameters, n_jobs=-1, verbose=1)
clf.fit(X_scaled,y)

print("Best score: %0.3f" % clf.best_score_)
print("Best params:",  clf.best_params_)



#eval_pred(clf, X, y, K=3)

Fitting 3 folds for each of 56 candidates, totalling 168 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 168 out of 168 | elapsed:   43.4s finished


[LibSVM][LibSVM]Best score: 0.914
Best params: {'svm__C': 0.10000000000000001, 'pf__degree': 2, 'feature_selection__threshold': 0.72857142857142865}


In [7]:
# normalise X_test data

clf.grid_scores_.sort(key=lambda x: x.mean_validation_score,
    reverse=True)
print(clf.grid_scores_[0:10])

X_test_scaled = scaler.transform(X_test)
print (clf.predict(X_test_scaled))

[mean: 0.91400, std: 0.00993, params: {'svm__C': 0.10000000000000001, 'pf__degree': 2, 'feature_selection__threshold': 0.72857142857142865}, mean: 0.91000, std: 0.01305, params: {'svm__C': 0.10000000000000001, 'pf__degree': 2, 'feature_selection__threshold': 0.81428571428571428}, mean: 0.91000, std: 0.00437, params: {'svm__C': 1.0, 'pf__degree': 2, 'feature_selection__threshold': 0.81428571428571428}, mean: 0.91000, std: 0.01235, params: {'svm__C': 1000.0, 'pf__degree': 2, 'feature_selection__threshold': 0.90000000000000002}, mean: 0.90900, std: 0.01169, params: {'svm__C': 1.0, 'pf__degree': 2, 'feature_selection__threshold': 0.72857142857142865}, mean: 0.90900, std: 0.00719, params: {'svm__C': 10.0, 'pf__degree': 2, 'feature_selection__threshold': 0.81428571428571428}, mean: 0.90900, std: 0.01506, params: {'svm__C': 0.10000000000000001, 'pf__degree': 2, 'feature_selection__threshold': 0.90000000000000002}, mean: 0.90900, std: 0.01361, params: {'svm__C': 100.0, 'pf__degree': 2, 'featur

In [172]:
sub = pd.read_csv("sample.csv")
sub['y'] = clf.predict(X_test_scaled)
sub.head()
sub.to_csv('test_basic.csv', index = False)